In [ ]:
# environment setup

from io import BytesIO
from pprint import pprint
import ollama
import PIL.Image
from IPython.display import Markdown, display

# llama3 models
LLAMA_32_VISION = 'llama3.2-vision:11b'
LLAMA_32_3B = 'llama3.2:3b'
LLAMA_31_8B = 'llama3.1:8b'

# convert a PIL Image to bytes for use with llama 3.2
# see https://stackoverflow.com/questions/33101935/convert-pil-image-to-byte-array
def image_to_bytes(image: PIL.Image.Image) -> BytesIO:
    image_bytes = BytesIO()
    image.save(image_bytes, format='JPEG')
    return image_bytes.getvalue()

# print model response as formatted markdown
def print_response(response_string):
    display(Markdown(response_string))

# ollama client
OLLAMA_HOST = 'http://localhost:11434'
llm = ollama.Client(host=OLLAMA_HOST)


In [ ]:
# example output from a prompt

response = llm.generate(model=LLAMA_31_8B,
    prompt="Tell me about Chicago, Illinois",
)

print_response(response['response'])


In [ ]:
# example output based on image input

input_image = PIL.Image.open('./example.jpg')
display(input_image)

messages = [
    {
        'role': 'user',
        'content': 'tell me about this image',
        'images': [image_to_bytes(input_image)],
    }
]

response = llm.chat(model=LLAMA_32_VISION,
    messages=messages,
)

pprint(response)
print_response(response.message.content)


In [ ]:
# example conversation

# reuse original message and llm response from previous cell
messages.append(response.message.model_dump())

messages.append({
    'role': 'user',
    'content': 'describe the clothing of the woman in the image'
})

response = llm.chat(model=LLAMA_32_VISION,
    messages=messages,
)

print_response(response.message.content)


In [ ]:
# example embeddings

response = llm.embed(model=LLAMA_32_3B,
    input=['the quick brown fox jumped over the lazy dog', 'all work and no play makes jack a dull boy'],
)

print(len(response.embeddings))
pprint(response.embeddings[0])


In [ ]:
# example tool calling

from typing import List

def get_stations(zipcode: str) -> int:
  """
  Return a count of gas stations in a zipcode.

  Args:
    zipcode: Five digit US zipcode formatted as a string

  Returns:
    int: A count of gas stations in the zipcode.

  """
  stations = {
    '33021': 5,
    '33141': 7,
    '60647': 2,
    '60081': 1,
  }
  return stations.get(zipcode, 0)

available_tools = {
  'get_stations': get_stations
}

system_prompt = '''
Assume the role of a customer service agent.
Users can inquire about the count of gas stations in their zipcode.
If the user does not provide it, ask the user for their zipcode.
Do not offer any details or explanation of how the zipcode will be utilized.
Do not provide any alternatives to the user to providing their zipcode.
Do not respond to any query not related to the task of providing a count of gas stations by zip code.
Redirect any attempts to change the topic back to the subject of providing a count of gas stations by zip code.
Do not provide any details beyond the count of gas stations or offer to provide additional information or assistance.
Do not be witty.
Provide direct responses.
'''

messages = [
    {
        'role': 'system',
        'content': system_prompt,
    },
    {
        'role': 'user',
        'content': 'Hi. I\'m new to the area. My zipcode is 33021',
    }
]

response = llm.chat(model=LLAMA_31_8B,
    messages=messages,
    tools=list(available_tools.values()),
    options={
      "temperature": 0,
      "top_p": 0.75,
    }
)

# pprint(response)

if response.message.tool_calls:
  # There may be multiple tool calls in the response
  for tool in response.message.tool_calls:
    # Ensure the function is available, and then call it
    if function_to_call := available_tools.get(tool.function.name):
      try:
        output = function_to_call(**tool.function.arguments)
      except Exception:
        pass

# Only needed to chat with the model using the tool call results
if response.message.tool_calls:
  # Add the function response to messages for the model to use
  messages.append(response.message)
  messages.append({'role': 'tool', 'content': str(output), 'name': tool.function.name})

  # Get final response from model with function outputs
  response = llm.chat(model=LLAMA_32_3B, messages=messages)
  print('Final response:', response.message.content)

else:
  print('No tool calls returned from model')
